在本章节，我们将同时加载chatglm3 base + finetune_lora_chatglm3。完成一个由微调后的chatglm3充当agent，做下游任务的例子。
我们使用的是一个文本情感分类的数据集，数据集格式为alpca格式。
finetun脚本基于finetune_basemodel_demo。

chatglm3 base作为意图识别模块，作为路由对下游的tools进行调用。

同时使用chatglm3本身作为一个embedding模块，完成知识库的检索召回（召回模块解耦，可替换为其他模型。）

In [ ]:
# 导入必要的包
from langchain.agents import AgentExecutor
from langchain.llms import AzureOpenAI
import openai
from llm import ChatGLM3
from tools import SearchTool, Text_classification_Tool
from agent import IntentAgent
import copy
import os

In [ ]:
# google search api ley
GOOGLE_API_KEY = ""
GOOGLE_CSE_ID = ""

In [ ]:
model_path = "THUDM/chatglm3-6b-base"
llm_base = ChatGLM3(model_path=model_path) # base chatglm3模型
llm_text_cls = ChatGLM3(model_path=model_path) # finetune好的chatglm3模型

In [ ]:
# 模型加载
llm_base.load_model()
llm_text_cls.load_model_from_checkpoint(checkpoint="text_classification")

In [ ]:
# 下游可路由的工具列表
tools = [SearchTool(llm=llm_ori, google_api_key=GOOGLE_API_KEY, google_cse_id=GOOGLE_CSE_ID),
         Text_classification_Tool(llm=llm_text_cls)]

In [ ]:
# 意图识别agent使用chatglm3 base充当
agent = IntentAgent(tools=tools, llm=llm_base)
agent_exec = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, max_iterations=1)
agent_exec.run("Input:今天丢失了我的钱包，里面有很重要的东西，心情很沮丧")